<a href="https://colab.research.google.com/github/Tanainan/Covid-19Thailand/blob/main/Data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
# import packages
from pandas import read_csv
import datetime
import pandas as pd
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_rows', 1000)
from tensorflow.keras.utils import to_categorical

# Import Data from Original Sources

## Cases and Policies

In [199]:
data = read_csv("https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/d98db2da901d5707dc3225ee163903005e7645f3/data/OxCGRT_latest.csv")

data = pd.DataFrame(data)

# select Thailand only
data = data.loc[data['CountryName'] == "Thailand"]

# select columns
data = data[["Date", "C1_School closing",
            "C2_Workplace closing",
            "C3_Cancel public events", "C4_Restrictions on gatherings",
            "C5_Close public transport", "C6_Stay at home requirements",
            "C7_Restrictions on internal movement", "C8_International travel controls",
            "H1_Public information campaigns", "ConfirmedCases", "ConfirmedDeaths"]]

# change column names
data = data.rename(columns = {"C1_School closing":"C1_School.closing",
            "C2_Workplace closing":"C2_Workplace.closing",
            "C3_Cancel public events":"C3_Cancel.public.events",
            "C4_Restrictions on gatherings":"C4_Restrictions.on.gatherings",
            "C5_Close public transport":"C5_Close.public.transport",
            "C6_Stay at home requirements":"C6_Stay.at.home.requirements",
            "C7_Restrictions on internal movement":"C7_Restrictions.on.internal.movement",
            "C8_International travel controls":"C8_International.travel.controls",
            "H1_Public information campaigns":"H1_Public.information.campaigns"})

# change date format
data.Date =  pd.to_datetime(data["Date"], format='%Y%m%d')

# reset index
data = data.reset_index(drop = True)

# replace NaN with 0 for ConfirmedCases and ConfirmedDeaths
data[["ConfirmedCases", "ConfirmedDeaths"]] = data[["ConfirmedCases", "ConfirmedDeaths"]].replace(np.nan, 0)

# remove NaN rows (bottom part)
data = data.dropna()

# add daily cases columns
data[["Cases_daily"]] = data['ConfirmedCases'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
data[["Deaths_daily"]] = data['ConfirmedDeaths'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])

# replace NaN with 0 for ConfirmedCases and Deaths_daily
data[["Cases_daily", "Deaths_daily"]] = data[["Cases_daily", "Deaths_daily"]].replace(np.nan, 0)


# start from 15/2/2020
data = data[data.loc[data['Date'] == "2020-02-15"].index[0]:]

# reset index
data = data.reset_index(drop = True)

# one-hot encoding
a = pd.DataFrame(to_categorical(data[["C1_School.closing"]]))
b = pd.DataFrame(to_categorical(data[["C2_Workplace.closing"]]))
c = pd.DataFrame(to_categorical(data[["C3_Cancel.public.events"]]))
d = pd.DataFrame(to_categorical(data[["C4_Restrictions.on.gatherings"]]))
e = pd.DataFrame(to_categorical(data[["C5_Close.public.transport"]]))
f = pd.DataFrame(to_categorical(data[["C6_Stay.at.home.requirements"]]))
g = pd.DataFrame(to_categorical(data[["C7_Restrictions.on.internal.movement"]]))
h = pd.DataFrame(to_categorical(data[["C8_International.travel.controls"]]))
j = pd.DataFrame(to_categorical(data[["H1_Public.information.campaigns"]]))






/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Apple Mobility

## Google Mobility

In [198]:
mob = read_csv("https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv")

mob = pd.DataFrame(mob)

# select only Thailand
mob = mob.loc[mob['country_region'] == "Thailand"]

# select columns
mob = mob[["date",
              "retail_and_recreation_percent_change_from_baseline",
              "grocery_and_pharmacy_percent_change_from_baseline",
              "parks_percent_change_from_baseline",
              "transit_stations_percent_change_from_baseline",
              "workplaces_percent_change_from_baseline",
              "residential_percent_change_from_baseline"]]

# change date format
mob.date =  pd.to_datetime(mob["date"])

# change column name
mob = mob.rename(columns = {'date':'Date'})

# reset index
mob = mob.reset_index(drop = True)

# start from 15/2/2020
mob = mob[mob.loc[mob['Date'] == "2020-02-15"].index[0]:]


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Combine Data

In [213]:
# whichever has fewer data
length = min(len(data), len(mob))

# Cases daily
data_i = pd.concat([data.Cases_daily[:length], 
                    a[:length],
                    b[:length],
                    c[:length],
                    d[:length],
                    e[:length],
                    f[:length],
                    g[:length],
                    h[:length],
                    j[:length],
                    mob[:length]], axis = 1)

# Deaths daily
data_d = pd.concat([data.Deaths_daily[:length], 
                    a[:length],
                    b[:length],
                    c[:length],
                    d[:length],
                    e[:length],
                    f[:length],
                    g[:length],
                    h[:length],
                    j[:length],
                    mob[:length]], axis = 1)
